# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [57]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
import pandas as pd
import string
import math
import sys

import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK
# Lemmatizace textu př.:  
# " ".join([token.lemma_ for token in lemmatizer(text)])

In [2]:
def lemmatize_text(text):
    return " ".join([token.lemma_ for token in lemmatizer(text)])

In [59]:
df = pd.read_csv('articles.csv', usecols=["title", "text"])

df['title'] = df['title'].str.replace('[^\w\s]','').str.lower()
df['text'] = df['text'].str.replace('[^\w\s]','').str.lower()
df['text'] = df['text'].str.replace('\s\s+',' ')
df['title'] = df['title'].str.replace('\s\s+',' ')

df['title'] = df['title'].apply(lemmatize_text)
df['text'] = df['text'].apply(lemmatize_text)


In [60]:
# Instalace spaCy z Jupyter Notebooku
#!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
#!{sys.executable} -m spacy download en

## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [50]:
def inverted_index(document):
    dic = {}
    for id_doc, row in enumerate(document):
        words = row.split(" ")
        for idx_word, word in enumerate(words):
            if word in dic.keys():
                if id_doc in dic[word]:
                    continue
                dic[word].append(id_doc)
            else:
                dic[word] = [id_doc]
    return dic

In [51]:
inv_title = inverted_index(df['title'].tolist())
inv_text = inverted_index(df['text'].tolist())

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [52]:
def tf_idf(querry,documents,df):
    scores = {}
    M = len(documents)
    words = querry.split(" ")
    c_wq = [words.count(w) for w in words]
    for idx_d,d in enumerate(documents.tolist()):
        score = 0
        for idx_q, word in enumerate(words):
            score += c_wq[idx_q] * d.count(word) * math.log((M+1)/len(df[word]))
        scores[idx_d] = score
    return scores

In [53]:
scores = tf_idf("coursera vs udacity machine learning", df['title'],inv_title)

## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [56]:
alpha = 0.7
querry = "coursera vs udacity machine learning"
scores_title = tf_idf(querry, df['title'],inv_title)
scores_text = tf_idf(querry, df['text'],inv_text)
scores = {}
df['scores'] = None
for idx, row in df.iterrows():
    df['scores'][idx]=alpha*scores_title[idx]+(1-alpha)*scores_text[idx]
df.sort_values('scores', ascending=False)

,title,text,scores
276,coursera vs udacity for machine learning hacke...,2018 be an exciting time for student of machin...,43.081344
19,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,33.907372
67,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,33.907372
99,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,33.907372
143,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,33.907372
...,...,...,...
267,what be the good intelligent chatbot or ai cha...,how do we define the intelligence of a chatbot...,0.0
130,selfdrive car and the trolley problem tanay ja...,google recently announce that their selfdrive ...,0.0
40,build a smart home feed pinter engineering medium,chris pinchak pinter engineer discovery \n the...,0.0
105,the evolve role of business analytic frank dia...,an old post that seem to be get a lot of atten...,0.0
